 <font size = +3.5><b><u><center> Phase 2 Project Notebook </center></u></b></font>

# Business Understanding

 * As a Construction Consulting firm, Royal Advisory helps new home renovation businesses establish themselves in the Seattle market.
 <br>  
  
  * With the housing market is cooling rapidly in Seattle, homeowners are increasingly opting to invest in remodeling their current home instead of buying new properties. Given this [increase in demand](https://www.king5.com/article/money/markets/real-estate/seattle-home-renovation-housing-cooldown/281-71cb28a8-1bd5-4187-955f-debad072fe53), now is the perfect time for a home renovation company to move into the Seattle area. 
  <br>  
  
  * A new home renovation company, Home Remedies, hired us to recommend the best services to offer to clients looking to increase the sale value of their homes. 
 <br>

## Types of Remodeling Projects

 * First, based on [industry definitions](https://www.homeadvisor.com/cost/additions-and-remodels/), we identifed three major components of home remodeling projects:
     1. **Home Renovations:** Updating the quality of the house's building material
     2. **Home Remodeling:** Increasing the size of your home, either through extensions or additions:
         2a. **Home Extensions:** Increasing the size of existing section of the house
         2b. **Home Additions:** Adding a new section to your home
     
 * We were then guided by these business questions:

## Business Questions: 

1. For all clients interested in increasing the area of their house (either through *home extensions* or *home additions*, where are the best areas to add square footage?
    - Which areas of the house is increased area associated with increased sale value?
    - Our model included these common home spaces: Above-Ground Living Space (e.g. rooms, kitchen, hallways), Basement, Garage, Patio, Yard

<br>  
2. How much does build quality contribute to the sale price of a house? Should clients add build quality renovation projects to their remodeling projects to increase their house's value?

<br>  
<br>  

3. For clients specifically interested in *home additions*, what are the best structures to add to increase the sale value of their house?
    - What types of home additions contribute to a higher sale price?

***

## Data Source & Relevance to Business Case

<font color = green><b><u> Rubric: Data Understanding pt 1 </u></b>
    
1. Describe the data sources and explain why the data are suitable for the project
2. Present the size of the dataset </font>

* Data from King County: all house sale in 2021 (n = 30155)

* Current data allows us to see what *features of a house* contributed to house sale price in 2021
   * From this, we can infer what types of house changes could increase future house sale price

# Data Preparation

<font color = green><b><u> Rubric: Data Preparation </u></b></font>

1. Instructions or code needed to get and prepare the raw data for analysis
2. Code comments and text to explain what your data preparation code does
3. Valid justifications for why the steps you took are appropriate for the problem you are solving

## Import Packages & Data

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import statsmodels.api as sm
import seaborn as sns
from scipy import stats

%matplotlib inline

In [4]:
# Collect entire data 

data = pd.read_csv('data/kc_house_data.csv')

## Exploring Variables in Dataset

In [ ]:
# Checkout all data columns & spot Null values

data.info()

## Identifying Features of Interest to Answer Current Questions

<font color = green><b><u> Rubric: Data Understanding pt2 </u></b>
    
2. descriptive statistics for all features used in the analysis
3. Justify the inclusion of features based on their properties and relevance for the project </font>

Features of Interest for each question:

Columns to Exclude:
    * We excluded variables that a home remodeling company *could not change* -- not useful information for our client

<font color = blue> <b> Move to Conclusion Section: </b>
    <br>  
    * Some of these variables could account for price valuation (such as location) <br>  
    * In future analyses, we could improve our models by including some of these as control variables </font>

In [ ]:
# Dropping columns not related to current questions/analyses

df = data.drop(['id', 'date', 'view', 'waterfront', 'greenbelt', 'nuisance', 'yr_built','yr_renovated', 'address', 'lat', 'long'], axis=1)

## Defining & Excluding Outliers

* We want to look at typical qualities of houses: only cases relevant to our client
    * Data indicates many outliers: 
        * Rare to encounter these clients with houses fitting these cases
        * Likely to have different goals/priorities than target audience

# Modeling

Runs and interprets a simple, baseline model for comparison

Introduces new models that improve on prior models and interprets their results

Explicitly justifies model changes based on the results of prior models and the problem context

Explicitly describes any improvements found from running new models

# Results

Explicitly identify the features and interpret their coefficients in plain language

Explain the implications of results for the real-world problem the project aims to address

Identify specific actions that stakeholders should take to leverage these results
